# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [ ]:
# Example reading the SQL database from here

# from dotenv import load_dotenv
# from sqlalchemy import create_engine
# import pandas as pd

# # load the .env file variables
# load_dotenv()

# connection_string = f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}?autocommit=true"
# engine = create_engine(connection_string)
# engine.connect()

# dataframe = pd.read_sql("Select * from books;", engine)
# print(dataframe.describe())

In [ ]:
# Example importing the CSV here

# dataframe = pd.read_csv('../path/to/file.csv')
# dataframe.describe()

In [ ]:
import os
import requests
import tweepy
import pandas as pd
from dotenv import load_dotenv

# load the .env file variables
load_dotenv()

consumer_key = os.env.get("CONSUMER_KEY")
consumer_secret = os.env.get("CONSUMER_SECRET")
bearer_token = os.env.get("BEARER_TOKEN")


# your app code here
client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret,
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

query = '#100daysofcode (pandas OR python) -is:retweet'

tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id','created_at','lang'],
                                     max_results=100)

tweets_dict = tweets.json() 

In [ ]:
import os
import requests 
import tweepy
import pandas as pd 
from dotenv import load_dotenv


# load the .env file variables
load_dotenv()

consumer_key = os.env.get("CONSUMER_KEY")
consumer_secret = os.env.get("CONSUMER_SECRET")
bearer_token= os.env.get("BEARER_TOKEN")



# Creando cliente de Twitter

client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret,
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

# Definiendo el query para Tweeter

query = '#100daysofcode (pandas OR python) -is:retweet'      

tweets = client.search_recent_tweets(query=query, 
                                    tweet_fields=['author_id','created_at','lang'],
                                     max_results=100)

In [ ]:
tweets

In [ ]:
tweets_dict = tweets.json() 

In [ ]:
list(tweets_dict)

In [ ]:
tweets_data = tweets_dict['data'] 
df = pd.json_normalize(tweets_data)
df

In [ ]:
tweets_data

In [ ]:
list(tweets_data[0])